In [1]:
import sys
sys.path.append("../")

In [4]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
from glob import glob
from tqdm import tqdm
import argparse
import os
import random
from termcolor import colored

import numpy as np
import torch
import torch.backends.cudnn as cudnn
import pandas as pd

In [6]:
from model_api import (
    # load_config,
    load_model, setup_seeds, ask_about_video,
)

In [9]:
model, vis_processor, args, chat = load_model()

Initializing Chat
Loading VIT


INFO:root:freeze vision encoder


Loading VIT Done
Loading Q-Former
use text input for Qformer


INFO:root:load checkpoint from /work/piyush/pretrained_checkpoints/LargeModels/TimeChat/instruct_blip_vicuna7b_trimmed.pth
INFO:root:freeze Qformer
INFO:root:Loading Q-Former Done
INFO:root:Loading LLAMA Tokenizer
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Using pad_token, but it is not set yet.
INFO:root:Loading LLAMA Model
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:27<00:00, 13.53s/it]
INFO:root:use gradient checkpointing for LLAMA
INFO:root:Loading 

trainable params: 0 || all params: 6,771,970,048 || trainable%: 0.0


INFO:root:video_Qformer is frozen


Load first Checkpoint: /work/piyush/pretrained_checkpoints/LargeModels/TimeChat//TimeChat-7b/timechat_7b.pth
Initialization Finished


**Load data**

In [10]:
# Directory containing all videos
video_dir = "/scratch/shared/nfs2/piyush/datasets/ViLMA/videos"

# Directory containing metadata
metad_dir = "/users/piyush/projects/ViLMA/data"


def load_json(path: str) -> dict:
    """Helper to load json file"""
    import json
    with open(path, 'rb') as f:
        data = json.load(f)
    return data

In [11]:
main_task = "change-state"
sub_task = "action"
task_file = os.path.join(
    metad_dir, f"{main_task}-{sub_task}.json"
)
assert os.path.join(task_file)
task_data = load_json(task_file)
len(task_data)

624

In [12]:
df = pd.DataFrame(task_data).T
df.shape

(624, 18)

In [13]:
# Add video ID to each row

def get_video_id(item):
    from_yt = ['RareAct', 'VidSitu', 'youcook2', 'coin']
    # find the full path
    dataset = item['dataset']
    video_file = item['video_file']
    # video_path = None
    if dataset == 'QUVA':
        normalized = item.get('normalized')
        assert normalized
        # video_dir = osp.join(self.quva_dir, 'normalized_videos')
        # video_path = osp.join(video_dir, video_file)
        video_id = video_file
    elif dataset == 'something-something-v2':
        # video_dir = self.something_something_dir
        # video_path = osp.join(video_dir, f'{item["dataset_idx"]}.webm')
        video_id = item["dataset_idx"]
    elif dataset == 'star':
        # video_dir = self.star_dir
        # video_path = osp.join(video_dir, f"{video_file}.mp4")
        video_id = video_file
    elif dataset in from_yt:
        # video_dir = self.youtube_dir
        # video_path = osp.join(video_dir, f'{item["youtube_id"]}.mp4')
        video_id = item["youtube_id"]
    else:
        raise NotImplementedError('Not implemented yet.')
    return video_id

video_ids = []
for i in range(len(df)):
    row = df.iloc[i].to_dict()
    video_id = get_video_id(row)
    video_ids.append(video_id)
df["video_id"] = video_ids

df.shape, df.video_id.isnull().sum()

((624, 19), 0)

In [14]:
def search_video_path(video_dir, video_id):
    paths = glob(os.path.join(video_dir, f"{video_id}.*"))
    assert len(paths) in [0, 1]
    if len(paths) == 0:
        return None
    else:
        path = paths[0]
        return path


df["video_path"] = df["video_id"].apply(
    lambda x: search_video_path(video_dir, x)
)
df.shape, df.video_path.isnull().sum()

((624, 20), 6)

In [15]:
subdf = df[df.video_path.apply(lambda x: os.path.exists(x) if x is not None else False)].copy()
subdf.shape

(618, 20)

In [16]:
# Test on a sample row
i = 0
row = subdf.iloc[i].to_dict()
video_path = row["video_path"]

caption = row["caption"].lower()
foil = row["foils"][0].lower()

randomise_options = True
enum_options = ["(a)", "(b)"]
if randomise_options:
    if np.random.uniform() < 0.5:
        text_options = [caption, foil]
        correct_answer = f"{enum_options[0]} {caption}"
    else:
        text_options = [foil, caption]
        correct_answer = f"{enum_options[1]} {caption}"
user_message = "Given this video, you have to select which is the option "\
    "that correctly describes the video: "\
    f"{enum_options[0]} {text_options[0]} "\
    f"{enum_options[1]} {text_options[1]} "\
    f"You have to only answer {enum_options[0]} or {enum_options[0]}."

model_answer = ask_about_video(chat, video_path, user_message)
print(model_answer)

The given query happens in 567, 1135, 2271, 3406, and 3974 frames.


In [20]:
correct_answer in model_answer

False

In [21]:
def check_row(row, verbose=False, randomise_options=True, enum_options=["(a)", "(b)"]):
    """Checks a single row."""

    video_path = row["video_path"]    
    caption = row["caption"].lower()
    foil = row["foils"][0].lower()
    
    if randomise_options:
        if np.random.uniform() < 0.5:
            text_options = [caption, foil]
            correct_answer = f"{enum_options[0]} {caption}"
        else:
            text_options = [foil, caption]
            correct_answer = f"{enum_options[1]} {caption}"
    else:
        text_options = [caption, foil]
        correct_answer = f"{enum_options[0]} {caption}"

    user_message = "Given this video, you have to select which is the option "\
        "that correctly describes the video: "\
        f"{enum_options[0]} {text_options[0]} "\
        f"{enum_options[1]} {text_options[1]} "\
        f"You have to only answer {enum_options[0]} or {enum_options[0]}."
    
    model_answer = ask_about_video(chat, video_path, user_message)
    flag = correct_answer in model_answer

    if verbose:
        print("QUESTION: ", user_message)
        print("VIDEO: ", video_path)
        print("MODEL ANSWER: ", model_answer)
        print("IDEAL ANSWER: ", correct_answer)

    return flag


from tqdm import tqdm

def tqdm_iterator(items, desc=None, bar_format=None, **kwargs):
    tqdm._instances.clear()
    iterator = tqdm(
        items,
        desc=desc,
        bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}',
        **kwargs,
    )
    tqdm._instances.clear()
    return iterator

In [ ]:
iterator = tqdm_iterator(range(len(subdf)), desc="Evaluating on entire dataset")
randomised_flags = []
nonrandomised_flags = []
failed = []
for i in iterator:
    row = subdf.iloc[i].to_dict()
    try:
        flag = check_row(row, verbose=False, randomise_options=True)
        flag_ = check_row(row, verbose=False, randomise_options=False)
    except:
        # Failed on this video
        failed.append(i)
    randomised_flags.append(flag)
    nonrandomised_flags.append(flag_)
    # break
randomised_flags = np.array(randomised_flags).astype(int)
nonrandomised_flags = np.array(nonrandomised_flags).astype(int)
print("Accuracy (with randomised options): ", np.mean(randomised_flags))
print("Accuracy (without randomised options): ", np.mean(nonrandomised_flags))

Evaluating on entire dataset:   1%|          | 5/618 [01:15<1:48:57, 10.67s/it]                                                                                                   

In [26]:
print("Accuracy (with randomised options): ", np.mean(randomised_flags))
print("Accuracy (without randomised options): ", np.mean(nonrandomised_flags))

Accuracy (with randomised options):  0.2313915857605178
Accuracy (without randomised options):  0.42394822006472493
